# Tensor builder walkthrough


PixelPrism exposes the variadic ``build_tensor`` operator to assemble tensors from scalar math expressions. This notebook highlights how constants, variables, and derived expressions can be interleaved while targeting vectors or higher-rank arrays.


In [1]:

import pixelprism.math as pm
import pixelprism.math.functional as F
from pixelprism.math import DType

scalar_alpha = pm.const("builder_scalar_alpha", data=1.5, dtype=pm.DType.FLOAT32)
scalar_beta = pm.const("builder_scalar_beta", data=-2.0, dtype=pm.DType.FLOAT64)
scalar_gamma = pm.const("builder_scalar_gamma", data=3, dtype=pm.DType.INT32)

builder_variable = pm.var("builder_variable", dtype=pm.DType.FLOAT32, shape=())
pm.set_value(builder_variable.name, 0.25)

def inspect(expr):
    tensor = expr.eval()
    print(f"{expr.name}: shape={tensor.shape.dims}, dtype={tensor.dtype}")
    print(tensor.value)
    return tensor
# end def inspect

### Building simple vectors
The operator defaults to a 1-D tensor whose length matches the number of provided elements.

In [2]:
basic_vector = F.build_tensor([scalar_alpha, scalar_beta, 7])
inspect(basic_vector)

tensor([builder_scalar_alpha, builder_scalar_beta, constant_4]): shape=(3,), dtype=DType.FLOAT64
[ 1.5 -2.   7. ]


tensor([1.5, -2.0, 7.0], dtype=DType.FLOAT64, shape=(3,), mutable=True)

### Mixing variables and expressions
Every entry can be a variable or a composed expression. Updating the context immediately impacts the built tensor.

In [3]:
pm.set_value(builder_variable.name, 1.75)
expression_vector = F.build_tensor(
    [
        builder_variable,
        scalar_alpha + scalar_beta,
        F.mul(builder_variable, scalar_gamma),
        scalar_gamma + 0.5,
    ]
)
inspect(expression_vector)


tensor([builder_variable, builder_scalar_alpha + builder_scalar_beta, builder_variable * builder_scalar_gamma, builder_scalar_gamma + constant_8]): shape=(4,), dtype=DType.FLOAT64
[ 1.75 -0.5   5.25  3.5 ]


tensor([1.75, -0.5, 5.25, 3.5], dtype=DType.FLOAT64, shape=(4,), mutable=True)

### Multi-dimensional tensors
Passing ``shape`` reshapes the flat buffer, enabling matrices or higher-order tensors.

In [4]:
pm.set_value(builder_variable.name, -0.25)
matrix_example = F.build_tensor(
    [
        scalar_alpha,
        builder_variable,
        scalar_beta,
        builder_variable + scalar_gamma,
    ],
    shape=(2, 2),
)
inspect(matrix_example)


tensor([builder_scalar_alpha, builder_variable, builder_scalar_beta, builder_variable + builder_scalar_gamma]) @ (2, 2): shape=(2, 2), dtype=DType.FLOAT64
[[ 1.5  -0.25]
 [-2.    2.75]]


tensor([[1.5, -0.25], [-2.0, 2.75]], dtype=DType.FLOAT64, shape=(2, 2), mutable=True)

### Larger layouts and dtype promotion
Shapes of any rank are supported so long as the flat element count matches. Mixed dtypes automatically promote to the highest precision involved.

In [5]:
pm.set_value(builder_variable.name, 0.0)
volume = F.build_tensor(
    [
        scalar_alpha,
        builder_variable,
        scalar_beta,
        scalar_gamma,
        builder_variable + scalar_gamma,
        scalar_alpha - scalar_beta,
        0.0,
        builder_variable - 1.0,
    ],
    shape=(2, 2, 2),
)
inspect(volume)


tensor([builder_scalar_alpha, builder_variable, builder_scalar_beta, builder_scalar_gamma, builder_variable + builder_scalar_gamma, builder_scalar_alpha - builder_scalar_beta, constant_17, builder_variable - constant_15]) @ (2, 2, 2): shape=(2, 2, 2), dtype=DType.FLOAT64
[[[ 1.5  0. ]
  [-2.   3. ]]

 [[ 3.   3.5]
  [ 0.  -1. ]]]


tensor([[[1.5, 0.0], [-2.0, 3.0]], [[3.0, 3.5], [0.0, -1.0]]], dtype=DType.FLOAT64, shape=(2, 2, 2), mutable=True)

## Vector and Matrix

In [6]:
pm.set_value(builder_variable.name, -0.5)
vector_example = F.vector(
    [
        scalar_alpha,
        builder_variable,
        scalar_beta,
        builder_variable + scalar_gamma,
    ]
)
inspect(vector_example)

vector([builder_scalar_alpha, builder_variable, builder_scalar_beta, builder_variable + builder_scalar_gamma]): shape=(4,), dtype=DType.FLOAT64
[ 1.5 -0.5 -2.   2.5]


tensor([1.5, -0.5, -2.0, 2.5], dtype=DType.FLOAT64, shape=(4,), mutable=True)

In [7]:
matrix_example = F.matrix(
    [
        [
            scalar_alpha,
            builder_variable
        ],
        [
            scalar_beta,
            builder_variable + scalar_gamma
        ],
    ]
)
inspect(matrix_example)

matrix(2x2): shape=(2, 2), dtype=DType.FLOAT64
[[ 1.5 -0.5]
 [-2.   2.5]]


tensor([[1.5, -0.5], [-2.0, 2.5]], dtype=DType.FLOAT64, shape=(2, 2), mutable=True)

In [8]:
full_example = F.full(
    shape=(2, 2),
    fill_value=scalar_alpha,
)
inspect(full_example)

full(builder_scalar_alpha, shape=(2, 2)): shape=(2, 2), dtype=DType.FLOAT32
[[1.5 1.5]
 [1.5 1.5]]


tensor([[1.5, 1.5], [1.5, 1.5]], dtype=DType.FLOAT32, shape=(2, 2), mutable=True)

In [9]:
zeros_example = F.zeros(
    shape=(2, 2),
    dtype=pm.DType.FLOAT64,
)
inspect(zeros_example)

zeros((2, 2)): shape=(2, 2), dtype=DType.FLOAT64
[[0. 0.]
 [0. 0.]]


tensor([[0.0, 0.0], [0.0, 0.0]], dtype=DType.FLOAT64, shape=(2, 2), mutable=True)

In [10]:
ones_example = F.ones(
    shape=(2, 2),
    dtype=pm.DType.INT32,
)
inspect(ones_example)

ones((2, 2)): shape=(2, 2), dtype=DType.INT32
[[1 1]
 [1 1]]


tensor([[1, 1], [1, 1]], dtype=DType.INT32, shape=(2, 2), mutable=True)

In [11]:
mat_mul_example = zeros_example @ ones_example
inspect(mat_mul_example)

zeros((2, 2)) @ ones((2, 2)): shape=(2, 2), dtype=DType.FLOAT64
[[0. 0.]
 [0. 0.]]


tensor([[0.0, 0.0], [0.0, 0.0]], dtype=DType.FLOAT64, shape=(2, 2), mutable=True)

In [12]:
eye_example = F.eye(
    rows=2,
    cols=2,
    dtype=pm.DType.FLOAT64,
)
inspect(eye_example)

eye(2x2): shape=(2, 2), dtype=DType.FLOAT64
[[1. 0.]
 [0. 1.]]


tensor([[1.0, 0.0], [0.0, 1.0]], dtype=DType.FLOAT64, shape=(2, 2), mutable=True)

In [13]:
mat_mul_example = eye_example @ ones_example + builder_variable
inspect(mat_mul_example)

eye(2x2) @ ones((2, 2)) + builder_variable: shape=(2, 2), dtype=DType.FLOAT64
[[0.5 0.5]
 [0.5 0.5]]


tensor([[0.5, 0.5], [0.5, 0.5]], dtype=DType.FLOAT64, shape=(2, 2), mutable=True)

In [14]:
mat_mul_example

<MathNode #31 add float64 (2, 2) c:2>

In [15]:
mat_mul_example.children

(<MathNode #30 matmul float64 (2, 2) c:2>,
 variable(builder_variable, dtype=DType.FLOAT32, shape=()))

In [16]:
pm.set_value(builder_variable.name, 2.5)
print(mat_mul_example.eval())

tensor([[3.5, 3.5], [3.5, 3.5]], dtype=DType.FLOAT64, shape=(2, 2), mutable=True)


## Concatenate

In [17]:
concat_example = F.concatenate([mat_mul_example, ones_example], axis=0)
inspect(concat_example)

concatenate(eye(2x2) @ ones((2, 2)) + builder_variable, ones((2, 2))): shape=(4, 2), dtype=DType.FLOAT64
[[3.5 3.5]
 [3.5 3.5]
 [1.  1. ]
 [1.  1. ]]


tensor([[3.5, 3.5], [3.5, 3.5], [1.0, 1.0], [1.0, 1.0]], dtype=DType.FLOAT64, shape=(4, 2), mutable=True)

## HStack and VStack

In [18]:
hstack_example = F.hstack([mat_mul_example, ones_example])
inspect(hstack_example)

hstack(eye(2x2) @ ones((2, 2)) + builder_variable, ones((2, 2))): shape=(2, 4), dtype=DType.FLOAT64
[[3.5 3.5 1.  1. ]
 [3.5 3.5 1.  1. ]]


tensor([[3.5, 3.5, 1.0, 1.0], [3.5, 3.5, 1.0, 1.0]], dtype=DType.FLOAT64, shape=(2, 4), mutable=True)

In [19]:
vstack_example = F.vstack([mat_mul_example, ones_example])
inspect(vstack_example)

vstack(eye(2x2) @ ones((2, 2)) + builder_variable, ones((2, 2))): shape=(4, 2), dtype=DType.FLOAT64
[[3.5 3.5]
 [3.5 3.5]
 [1.  1. ]
 [1.  1. ]]


tensor([[3.5, 3.5], [3.5, 3.5], [1.0, 1.0], [1.0, 1.0]], dtype=DType.FLOAT64, shape=(4, 2), mutable=True)

## FromFunction

In [20]:
body_expr = builder_variable + pm.var("i", pm.DType.INT32, pm.Shape.scalar()) + 2.0 * pm.var("j", pm.DType.INT32, pm.Shape.scalar())
from_function_example = F.from_function(shape=(2, 2), body=body_expr)

In [21]:
inspect(from_function_example)

from_function(shape=(2, 2)): shape=(2, 2), dtype=DType.FLOAT32
[[2.5 4.5]
 [3.5 5.5]]


tensor([[2.5, 4.5], [3.5, 5.5]], dtype=DType.FLOAT32, shape=(2, 2), mutable=True)

In [22]:
pm.get_value(builder_variable.name)

tensor(2.5, dtype=DType.FLOAT32, shape=(), mutable=True)

## Sparse COO

In [23]:
sparse_coo_example = F.sparse_coo(shape=(2, 2), indices=[(0, 0), (1, 1)], values=[1.0, 2.0])
inspect(sparse_coo_example)

sparse_coo(shape=(2, 2), nnz=2): shape=(2, 2), dtype=DType.FLOAT32
[[1. 0.]
 [0. 2.]]


tensor([[1.0, 0.0], [0.0, 2.0]], dtype=DType.FLOAT32, shape=(2, 2), mutable=True)

In [24]:
sparse_coo_var1 = pm.var("sparse_coo_var1", pm.DType.FLOAT32, pm.Shape.scalar())
sparse_coo_var2 = pm.var("sparse_coo_var2", pm.DType.FLOAT32, pm.Shape.scalar())
with pm.new_context():
    pm.set_value(sparse_coo_var1.name, 1.5)
    pm.set_value(sparse_coo_var2.name, 2.0)
    sparse_coo_example = F.sparse_coo(shape=(2, 2), indices=[(0, 0), (1, 1)], values=[sparse_coo_var1, sparse_coo_var2])
    inspect(sparse_coo_example)
# end with

sparse_coo(shape=(2, 2), nnz=2): shape=(2, 2), dtype=DType.FLOAT32
[[1.5 0. ]
 [0.  2. ]]


## Linspace and Logspace

In [25]:
start_var = pm.var("start_var", pm.DType.FLOAT32, pm.Shape.scalar())
stop_var = pm.var("stop_var", pm.DType.FLOAT32, pm.Shape.scalar())
num_var = pm.const("num_var", 10, pm.DType.INT32)
base_var = pm.var("base_var", pm.DType.FLOAT32, pm.Shape.scalar())
with pm.new_context():
    pm.set_value("start_var", -1)
    pm.set_value("stop_var", 2.0)
    pm.set_value("num_var", 10)
    pm.set_value("base_var", 10.0)
    linspace_expr = F.linspace(start_var, stop_var, num_var)
    inspect(linspace_expr)
    logspace_expr = F.logspace(start_var, stop_var, num_var, base=base_var)
    inspect(logspace_expr)
# end with

linspace(start_var, stop_var, 10): shape=(10,), dtype=DType.FLOAT32
[-1.         -0.6666667  -0.33333334  0.          0.33333334  0.6666667
  1.          1.3333334   1.6666666   2.        ]
logspace(start_var, stop_var, 10): shape=(10,), dtype=DType.FLOAT32
[  0.1          0.21544346   0.4641589    1.           2.1544347
   4.6415887   10.          21.544348    46.41589    100.        ]


## Map

In [26]:
map_expr = F.map_(
    tensor=F.linspace(start_var, stop_var, num_var),
    var_name="x",
    body=pm.var("x", DType.FLOAT32, pm.Shape.scalar()) * 2.0
)
with pm.new_context():
    pm.set_value("start_var", 0.0)
    pm.set_value("stop_var", 10.0)
    pm.set_value("num_var", 10)
    pm.set_value("base_var", 10.0)
    inspect(map_expr)
# end with

map(linspace(start_var, stop_var, 10), var=x): shape=(10,), dtype=DType.FLOAT32
[ 0.         2.2222223  4.4444447  6.6666665  8.888889  11.111111
 13.333333  15.555555  17.777779  20.       ]
